In [ ]:
# Write your imports here

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re
import nltk
import os

# Working with Text Lab
## Information retrieval, preprocessing, and feature extraction

In this lab, you'll be looking at and exploring European restaurant reviews. The dataset is rather tiny, but that's just because it has to run on any machine. In real life, just like with images, texts can be several terabytes long.

The dataset is located [here](https://www.kaggle.com/datasets/gorororororo23/european-restaurant-reviews) and as always, it's been provided to you in the `data/` folder.

### Problem 1. Read the dataset (1 point)
Read the dataset, get acquainted with it. Ensure the data is valid before you proceed.

How many observations are there? Which country is the most represented? What time range does the dataset represent?

Is the sample balanced in terms of restaurants, i.e., do you have an equal number of reviews for each one? Most importantly, is the dataset balanced in terms of **sentiment**?

In [ ]:
reviews = pd.read_csv('data\European Restaurant Reviews.csv')

reviews.shape

reviews.columns

In [ ]:
reviews['Country'].max()

Now we are going to clean up the date column in order to prepare it for datetime conversion and then we are going to extract the range.

In [ ]:
reviews['Review Date'] = reviews['Review Date'].str.extract(r'([A-Za-z]+\s+\d{4})', expand=False)


In [ ]:
reviews['Review Date'] = reviews['Review Date'].str.replace('Sept', 'Sep')

In [ ]:
reviews['Review Date'] = pd.to_datetime(reviews['Review Date'], format='%b %Y')


In [ ]:
reviews['Review Date'] = reviews['Review Date'].dt.to_period('M')

In [ ]:
min_date = reviews['Review Date'].min()
max_date = reviews['Review Date'].max()

print(f"Date range: {min_date} to {max_date}")


Next, focusing on sentiments. 

In [ ]:
restaurant_counts = reviews['Restaurant Name'].value_counts()

print(restaurant_counts)


In [ ]:
sentiment_counts = reviews['Sentiment'].value_counts()
print("\nReviews per sentiment:\n", sentiment_counts)

The dataset is unbalanced in terms of both restaurant reviews and sentiment.

### Problem 2. Getting acquainted with reviews (1 point)
Are positive comments typically shorter or longer? Try to define a good, robust metric for "length" of a text; it's not necessary just the character count. Can you explain your findings?

We are going to use word count, sentence count and average word length. Word count will show us tendencies in review length for positive and for negative reviews. Sentence count will represent similar insight, while word length will show us the complexity of the words used.

In [ ]:
reviews['Word Count'] = reviews['Review'].apply(lambda x: len(str(x).split()))
print(reviews[['Review', 'Word Count']].head())


In [ ]:
avg_word_count_per_sentiment = reviews.groupby('Sentiment')['Word Count'].mean()
print(avg_word_count_per_sentiment)


In [ ]:
sns.boxplot(x='Sentiment', y='Word Count', data=reviews)
plt.title('Distribution of Review Length by Sentiment')
plt.show()

The average word length being greater for positive reviews tells us that users perhaps tend to elaborate when giving a bad review, while they don't need to delve into specifics when giving good feedback.

In [ ]:
reviews['Sentence Count'] = reviews['Review'].apply(lambda x: len(nltk.sent_tokenize(x)))

In [ ]:
reviews['Average Word Length'] = reviews['Review'].apply(lambda x: np.mean([len(word) for word in re.findall(r'\w+', x)]))

In [ ]:
avg_sentence_count_per_sentiment = reviews.groupby('Sentiment')['Sentence Count'].mean()
print(avg_sentence_count_per_sentiment)

In [ ]:
avg_word_length_per_sentiment = reviews.groupby('Sentiment')['Average Word Length'].mean()
print(avg_word_length_per_sentiment)

### Problem 3. Preprocess the review content (2 points)
You'll likely need to do this while working on the problems below, but try to synthesize (and document!) your preprocessing here. Your tasks will revolve around words and their connection to sentiment. While preprocessing, keep in mind the domain (restaurant reviews) and the task (sentiment analysis).

### Problem 3. Top words (1 point)
Use a simple word tokenization and count the top 10 words in positive reviews; then the top 10 words in negative reviews*. Once again, try to define what "top" words means. Describe and document your process. Explain your results.

\* Okay, you may want to see top N words (with $N \ge 10$).

### Problem 4. Review titles (2 point)
How do the top words you found in the last problem correlate to the review titles? Do the top 10 words (for each sentiment) appear in the titles at all? Do reviews which contain one or more of the top words have the same words in their titles?

Does the title of a comment present a good summary of its content? That is, are the titles descriptive, or are they simply meant to catch the attention of the reader?

### Problem 5. Bag of words (1 point)
Based on your findings so far, come up with a good set of settings (hyperparameters) for a bag-of-words model for review titles and contents. It's easiest to treat them separately (so, create two models); but you may also think about a unified representation. I find the simplest way of concatenating the title and content too simplistic to be useful, as it doesn't allow you to treat the title differently (e.g., by giving it more weight).

The documentation for `CountVectorizer` is [here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html). Familiarize yourself with all settings; try out different combinations and come up with a final model; or rather - two models :).

### Problem 6. Deep sentiment analysis models (1 point)
Find a suitable model for sentiment analysis in English. Without modifying, training, or fine-tuning the model, make it predict all contents (or better, combinations of titles and contents, if you can). Meaure the accuracy of the model compared to the `sentiment` column in the dataset.

### Problem 7. Deep features (embeddings) (1 point)
Use the same model to perform feature extraction on the review contents (or contents + titles) instead of direct predictions. You should already be familiar how to do that from your work on images.

Use the cosine similarity between texts to try to cluster them. Are there "similar" reviews (you'll need to find a way to measure similarity) across different restaurants? Are customers generally in agreement for the same restaurant?

### \* Problem 8. Explore and model at will
In this lab, we focused on preprocessing and feature extraction and we didn't really have a chance to train (or compare) models. The dataset is maybe too small to be conclusive, but feel free to play around with ready-made models, and train your own.